1. Threads et GIL

Objectif : comprendre comment les threads permettent d'exécuter plusieurs tâches en parallèle du point de vue logique.

Points clés :
- Un thread partage la mémoire du processus principal.
- En Python, le GIL limite le parallélisme pour les tâches CPU-bound.
- Les threads restent utiles pour l'I/O (réseau, disque) car l'attente libère le GIL.

In [2]:
import threading
import time

# Exemple simple de thread : chaque thread exécute la même fonction avec un numéro différent.

def travail(numero):
    # Affiche le début du traitement pour ce thread.
    print(f'Démarrage du travail {numero}')
    # Simule une tâche bloquante (I/O) avec un temps d'attente.
    time.sleep(1)
    # Affiche la fin du traitement pour ce thread.
    print(f'Fin du travail {numero}')

# Liste qui va stocker les objets Thread.
threads = []
# Crée plusieurs threads numérotés.
for i in range(3):
    # Création d'un thread qui va exécuter la fonction travail.
    t = threading.Thread(target=travail, args=(i,))
    # Conserve une référence au thread pour pouvoir l'attendre.
    threads.append(t)
    # Démarre l'exécution du thread.
    t.start()

# Attendre la fin de tous les threads avant de continuer.
for t in threads:
    # join() bloque jusqu'à la fin du thread.
    t.join()

# Note: Python a le GIL (Global Interpreter Lock) qui empêche les threads CPU-bound d'être vraiment parallèles.

Démarrage du travail 0
Démarrage du travail 1
Démarrage du travail 2
Fin du travail 0
Fin du travail 1
Fin du travail 2


2. Utiliser plusieurs coeurs avec multiprocessing

Objectif : utiliser plusieurs processus pour contourner le GIL sur des tâches CPU-bound.

Points clés :
- Chaque processus a sa propre mémoire et son propre GIL.
- Le coût de création est plus élevé que pour un thread.
- Idéal pour les calculs intensifs.

In [3]:
from multiprocessing import Process, cpu_count

# Affiche le nombre de coeurs CPU disponibles.
print('\nNombre de coeurs disponibles:', cpu_count())

def travail_cpu(numero):
    # Indique le démarrage de la tâche CPU.
    print(f'Traitement CPU {numero} démarré')
    # Calcul intensif pour simuler une tâche CPU-bound.
    total = sum(i*i for i in range(10**6))
    # total n'est pas utilisé, mais force le calcul.
    print(f'Traitement CPU {numero} terminé')

# Liste qui va stocker les objets Process.
processes = []
# Crée plusieurs processus numérotés.
for i in range(2):
    # Chaque Process exécute travail_cpu dans un processus séparé.
    p = Process(target=travail_cpu, args=(i,))
    # Conserve une référence au processus pour pouvoir l'attendre.
    processes.append(p)
    # Démarre l'exécution du processus.
    p.start()

# Attendre la fin de tous les processus.
for p in processes:
    # join() bloque jusqu'à la fin du processus.
    p.join()


Nombre de coeurs disponibles: 20
Traitement CPU 0 démarré
Traitement CPU 1 démarré
Traitement CPU 0 terminéTraitement CPU 1 terminé



3. Programmation I/O asynchrone

Objectif : utiliser asyncio pour gérer plusieurs opérations I/O sans créer de threads.

Points clés :
- Une seule boucle d'événements orchestre des coroutines.
- Très efficace pour des milliers de connexions réseau.
- Ne convient pas aux calculs CPU lourds.

In [5]:
%pip install aiohttp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import nest_asyncio
# Permet d'exécuter asyncio dans un notebook déjà en boucle.
nest_asyncio.apply()
import asyncio

async def tache_io(numero):
    # Affiche le début de la coroutine.
    print(f'Tâche {numero} démarrée')
    # await libère la boucle d'événements pendant l'attente.
    await asyncio.sleep(1)  # simule une I/O non bloquante
    # Affiche la fin de la coroutine.
    print(f'Tâche {numero} terminée')

async def main():
    # Lancer plusieurs coroutines en parallèle logique.
    await asyncio.gather(*(tache_io(i) for i in range(3)))

# Exécute la boucle d'événements et la coroutine principale.
asyncio.run(main())

Tâche 0 démarrée
Tâche 1 démarrée
Tâche 2 démarrée
Tâche 0 terminée
Tâche 1 terminée
Tâche 2 terminée


4. Performances et éthique

Bonnes pratiques pour le scraping et l'automatisation réseau :
- Respecter les limites de requêtes (rate limiting).
- Ajouter des délais et mettre en cache pour réduire la charge.
- Vérifier le fichier robots.txt avant de collecter des données.

5. Utilisation d'une forme de cache

Objectif : éviter de re-télécharger des données déjà récupérées.

Points clés :
- Un cache réduit la latence et la charge réseau.
- Le stockage disque persiste entre exécutions.

In [8]:
import os
import pickle
import random

# Nom du fichier de cache.
cache_file = 'cache.pkl'

# Exemple simple de cache disque : on recharge si le fichier existe.
if os.path.exists(cache_file):
    # Ouvre le fichier de cache en lecture binaire.
    with open(cache_file, 'rb') as f:
        # Charge le dictionnaire du cache depuis le disque.
        cache = pickle.load(f)
else:
    # Si le cache n'existe pas, on part d'un dictionnaire vide.
    cache = {}

# Identifiant de ressource à mettre en cache.
url = 'https://example.com/data'
if url in cache:
    # Si la clé est présente, on réutilise la valeur.
    print('\nDonnées depuis le cache')
    data = cache[url]
else:
    # Sinon, on simule un téléchargement et on ajoute au cache.
    print('\nDonnées simulées et mise en cache')
    # Simule une réponse distante.
    data = {'value': random.randint(0,100)}
    # Sauvegarde dans le dictionnaire de cache.
    cache[url] = data
    # Écrit le cache mis à jour sur le disque.
    with open(cache_file, 'wb') as f:
        pickle.dump(cache, f)

# Affiche les données finales.
print('Données:', data)


Données simulées et mise en cache
Données: {'value': 70}


6. Introduire un délai aléatoire

Objectif : lisser la charge et éviter les accès trop rapides.

Point clé : un délai variable ressemble davantage à un comportement humain.

In [ ]:
import time

# Délai compris entre 1 et 3 secondes.
delay = random.uniform(1, 3)
# Affiche la durée de pause choisie.
print(f'Pause aléatoire de {delay:.2f} secondes')
# Met le programme en pause.
time.sleep(delay)


7. Vérifier le fichier robots.txt

Objectif : vérifier si un site autorise l'accès à une page.

Exemple d'un robot.txt simple :
```
User-agent: *
Disallow: /private/
```

Ce qui signifie que tous les agents (robots) sont interdits d'accéder à la section /private/ du site.

Point clé : robots.txt est une convention, pas un mécanisme de sécurité.

In [ ]:
import urllib.robotparser

# Crée un parseur pour le fichier robots.txt.
rp = urllib.robotparser.RobotFileParser()
# Charger les règles du site ciblé.
rp.set_url('https://jsonplaceholder.typicode.com/robots.txt')
# Télécharge et parse le robots.txt.
rp.read()
# Vérifie si un user-agent générique peut accéder à l'URL.
can_fetch = rp.can_fetch('*', 'https://jsonplaceholder.typicode.com/posts')
# Affiche le résultat de la vérification.
print(f'Peut-on scraper la page? {can_fetch}')

8. Exercices

1. Modifier le cache pour utiliser Redis (requiert installation redis-py et serveur Redis).
2. Comparer performance entre threading, multiprocessing et asyncio.
3. Implémenter un scraper respectueux qui lit robots.txt, utilise cache et délai aléatoire.

